In [44]:
import requests
import json
import codecs
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
api_uri ='https://sp.olx.com.br/vale-do-paraiba-e-litoral-norte/vale-do-paraiba/autos-e-pecas/carros-vans-e-utilitarios'

In [7]:
dummy_product_names = []
product_names = []

In [112]:
def get_dados(lista):
    dados = []
    for label in lista.find_all('li'):
        nome, price, desc, ano, fotos = '', '', '', '',''
        item = label.find('h2', {'class': 'OLXad-list-title'})
        if item:
            nome = item.text.strip()
            if nome:
                ano = nome.split('-')[-1]
        item = label.find('p', {'class': 'OLXad-list-price'})
        if item:
            price = item.text.strip().replace('R$', '').replace('.', '')
        item = label.find('p', {'class': 'text detail-specific'})
        if item:
            desc = item.text.strip().replace('\n', '').replace('\t', '')
        item = label.find('div', {'class': 'OLXad-list-image-counter'})
        if item:
            d = item.text.strip().split('\t')
            fotos = d[-1] if d else 0
        dados.append([nome, price, desc, ano, fotos])
    return dados

In [113]:
url = 'https://sp.olx.com.br/vale-do-paraiba-e-litoral-norte/vale-do-paraiba/autos-e-pecas/carros-vans-e-utilitarios'

In [119]:
lista_carros = []
for page in range(101):
    conteudo = requests.get(url, params={'o': 1}, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:55.0) Gecko/20100101 Firefox/55.0"}).content
    soup = BeautifulSoup(conteudo, "html.parser")
    lista = soup.find('ul', {'id': 'main-ad-list'})
    lista_carros.extend(get_dados(lista))

In [120]:
carros = pd.DataFrame(lista_carros, columns=['nome', 'price', 'desc', 'ano', 'fotos'])

In [121]:
carros.shape

(5454, 5)

In [122]:
carros['price'] = carros['price'].apply(lambda x: int(x) if x else 0)
carros['ano'] = carros['ano'].apply(lambda x: int(x) if x else 0)
carros['fotos'] = carros['fotos'].apply(lambda x: int(x) if x else 0)

In [123]:
carros.head()

,nome,price,desc,ano,fotos
0,Venda ou troca - 2012,22000,190.000 km | Câmbio: manual | Flex,2012,3
1,"Raridade! fiesta 1.6 completo só com 40.000,km...",24900,40.000 km | Câmbio: manual | Flex,2013,1
2,,0,,0,0
3,C3 tendance!!!impecável - 2013,31900,111.000 km | Câmbio: manual | Flex,2013,3
4,Polo 2003 gasolina 1.6 - 2003,15500,106.000 km | Câmbio: manual | Gasolina,2003,6
